# Large-Scale Morphology Analysis

In [ ]:
# load third-party Python modules
import javabridge
import bioformats as bf
import skimage
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import re
import os
import sys
import h5py
from sklearn.manifold import TSNE

javabridge.start_vm(class_path=bf.JARS)

In [ ]:
# load plate annotation
annot_df = pd.read_csv('../data/AML_trainset/drugannot.txt',
                      sep='\t')

In [ ]:
#plates = [f for f in os.listdir('../imgdata') if 'Plate' in f]
plates = ['180528_Plate3', '180528_Plate5',
          '180625_Plate3', '181109_Plate1',
          '181109_Plate3']

In [ ]:
platedir = os.path.join('../imgdata/', plates[0])

In [ ]:
dmso = annot_df[annot_df.Drug == 'DMSO'].reset_index(drop=True)
dmso_wells = dmso['well'].unique()

In [ ]:
imgdf = []
for w in dmso_wells:
    imgdf.append(pd.read_csv(os.path.join(platedir, w+'.csv')))

In [ ]:
imgdf = pd.concat(imgdf).reset_index(drop=True)
labels = imgdf[['class', 'file']]
imgdf = imgdf.drop(['class', 'file'], axis=1)
labels['well'] = labels['file'].replace(regex=r'f[0-9].+', value='')
labels['class'] = labels['class'].apply(lambda x: 'Viable' if x == 2 else 'Apoptotic')
labels = pd.merge(labels, dmso, on='well')

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X=imgdf)

In [ ]:
from bioimg.singlecell import scale_data, check_data
imgdf_scaled = scale_data(imgdf, scaler=scaler)

In [ ]:
check_data(imgdf_scaled)

In [ ]:
feat_subset = ['ch-Calcein-area',
               'ch-Calcein-eccentricity',
               'ch-Calcein-mean_intensity',
               'ch-Hoechst-area',
               'ch-Hoechst-eccentricity',
               'ch-Hoechst-mean_intensity',
               'ch-Lysosomal-area',
               'ch-Lysosomal-eccentricity',
               'ch-Lysosomal-mean_intensity']

In [ ]:
Xfeat = imgdf_scaled.loc[:,feat_subset]

In [ ]:
X_tsne = TSNE(n_components=2, random_state=21, perplexity=50).fit_transform(imgdf_scaled)

In [ ]:
X_df = pd.concat([pd.DataFrame(X_tsne, columns=['tsne1', 'tsne2']), labels], axis=1)

In [ ]:
X_df = pd.concat([X_df, Xfeat], axis=1)

In [ ]:
from bioimg.singlecell import plot_dimred
plot_dimred(X_df, 
            hue='Culture',
            style='class',
            title='DMSO control wells')

## Morphology analysis for viable cells
Subset only to viable cells:

In [ ]:
img_viab = imgdf.iloc[np.where(labels['class'] == 'Viable')[0],:]
scaler = StandardScaler().fit(img_viab)
img_viab = scale_data(img_viab, scaler=scaler).reset_index(drop=True)
labels_viab = (labels[labels['class'] == 'Viable'].
              reset_index(drop=True))

In [ ]:
X_tsne = TSNE(n_components=2, random_state=21, perplexity=30).fit_transform(img_viab)

In [ ]:
X_viab = pd.concat([pd.DataFrame(X_tsne, columns=['tsne1', 'tsne2']), labels_viab], axis=1)

In [ ]:
X_viab = pd.concat([X_viab, img_viab.loc[:,feat_subset]], axis=1)

In [ ]:
plot_dimred(X_viab, hue='Culture',
           title='Viable cells in control wells')
plt.legend(loc='lower right',
           bbox_to_anchor=(1.2,0.05))

In [ ]:
from bioimg.singlecell import facet_dimred
facet_dimred(X_viab, feat_subset=feat_subset,
            nrows=3, ncols=3)

Save the viable cells of DMSO wells before loading drug-treated wells:

In [ ]:
img_viab = imgdf.iloc[np.where(labels['class'] == 'Viable')[0],:]
X_ctrl = img_viab

Now load some drug wells:

In [ ]:
drug_sel = ['Tofacitinib', 'Midostaurin',
                   'Ganetespib', 'Lenalidomide',
                   'Pyridone 6', 'UMI-77',
                   'Bafilomycin A1', 
                   'Quizartinib', 'Hydroxychloroquine',
                   'Fludarabine', 'Vorinostat',
                   'Thioguanine', 'Nutlin 3a',
                   'Palbociclib', 'Carfilzomib',
                   'JQ1', 'Cytarabine',
                   'BAY61-3606', 'Venetoclax',
                   'Ixazomib']

In [ ]:
drugs = annot_df[np.isin(annot_df.Drug, drug_sel) ].reset_index(drop=True)
drug_wells = drugs['well'].unique()

In [ ]:
imgdf = []
for w in drug_wells:
    df = pd.read_csv(os.path.join(platedir, w+'.csv'))
    imgdf.append(df[df['class'] == 2])

In [ ]:
imgdf = pd.concat(imgdf).reset_index(drop=True)
labels = imgdf[['class', 'file']]
imgdf = imgdf.drop(['class', 'file'], axis=1)
labels['well'] = labels['file'].replace(regex=r'f[0-9].+', value='')
labels['class'] = labels['class'].apply(lambda x: 'Viable' if x == 2 else 'Apoptotic')
labels = pd.merge(labels, drugs, on='well')

In [ ]:
X_drug = scale_data(imgdf, scaler=scaler)
Xfeat = X_drug.loc[:,feat_subset]

In [ ]:
X_tsne = TSNE(n_components=2, random_state=21, perplexity=50).fit_transform(X_drug)

In [ ]:
X_df = pd.concat([pd.DataFrame(X_tsne, columns=['tsne1', 'tsne2']), labels], axis=1)
X_df = pd.concat([X_df, Xfeat], axis=1)

In [ ]:
plot_dimred(X_df, hue='Culture',
           title='Viable cells in drug-treated wells')

In [ ]:
drug_chunks = [drug_sel[i:i + 5] for i in range(0, len(drug_sel), 5)]

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=2,
                       figsize = (14,12))
sn.set(font_scale=1.2)
sn.set_style('white')
sn.despine()
for r in range(2):
    for c in range(2):
        sn.scatterplot(x = 'tsne1', y = 'tsne2', data=X_df[np.isin(X_df['Drug'], drug_chunks[r*2+c])],
               hue = 'Drug',
               s = 40, alpha = 0.8, ax=ax[r,c])
        ax[r,c].legend(loc='lower right', bbox_to_anchor=(1.4,0.7))
        ax[r,c].set_xlabel('TSNE 1')
        ax[r,c].set_ylabel('TSNE 2')
fig.subplots_adjust(wspace=0.5)

In [ ]:
facet_dimred(X_df, feat_subset=feat_subset,
            nrows=3, ncols=3)

In [ ]:
X_df.columns

In [ ]:
colsub = ['Culture'] +feat_subset
X_long = pd.melt(X_df[colsub], 
                 id_vars=['Culture'], 
                 value_vars=feat_subset,
                 var_name='feature', 
                 value_name='val')

In [ ]:
from bioimg.singlecell import facet_density
facet_density(X_long, feat_column='feature',
             ncols=3, hue='Culture')

In [ ]:
colsub = ['Drug', 'Culture'] +feat_subset
X_long = pd.melt(X_df[colsub], 
                 id_vars=['Drug', 'Culture'], 
                 value_vars=feat_subset,
                 var_name='feature', 
                 value_name='val')

In [ ]:
from bioimg.singlecell import facet_boxplot
facet_boxplot(X_long, x='Drug',
              y='val', feat_column='feature',
              ncols=3,
              nrows=3, hue='Culture')

In [ ]:
X_ctrl = X_ctrl.reset_index(drop=True)                     

In [ ]:
labels_viab = labels_viab.reset_index(drop=True)

In [ ]:
Xfeat = pd.concat([labels_viab['Culture'], X_ctrl], axis=1)

Check correlation structure of the features:

In [ ]:
from scipy.spatial.distance import pdist, squareform
def cordf(X):
    return squareform(1. - pdist(X, 'correlation'))

In [ ]:
# feature correlation
featcor = pd.DataFrame(cordf(X_ctrl.T),
                       index=X_ctrl.columns,
                       columns=X_ctrl.columns )

In [ ]:
from bioimg.singlecell import plot_heatmap
plot_heatmap(featcor)

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

In [ ]:
y=labels_viab['Culture'].values
y[y=='Mono-culture'] = 0
y[y=='Co-culture'] = 1
y = y.astype(int)

In [ ]:
clf = RandomForestClassifier(n_estimators=500,
                             max_depth=7,
                             random_state=3,
                             n_jobs=-1)
rfe = RFE(estimator=clf, n_features_to_select=500, step=50)
rfe.fit(X_ctrl, y)

In [ ]:
sel_features = X_ctrl.columns[rfe.get_support()].values

Run Mann-Whitney-Wilcoxon test:

In [ ]:
from scipy.stats import mannwhitneyu

In [ ]:
plt.hist((X_ctrl.apply(lambda x: mannwhitneyu(x[labels_viab.Culture == 'Mono-culture'],
                                     x[labels_viab.Culture == 'Co-culture'])[1])).values)

In [ ]:
pvaldf = pd.DataFrame(X_ctrl.apply(lambda x: mannwhitneyu(x[labels_viab.Culture == 'Mono-culture'],
                                     x[labels_viab.Culture == 'Co-culture'])[1]), columns=['pval'])

In [ ]:
pvaldf.head()